### Comprobación de GPU (linux)

In [2]:
!/usr/bin/nvidia-smi

Wed Oct 30 23:05:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     Off |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8              1W /   80W |     357MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

I0000 00:00:1730329541.139117    3437 gpu_device.cc:2022] Created device /device:GPU:0 with 4344 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


'/device:GPU:0'

In [5]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")
model.fuse() #model.summary 
class_names = model.model.names
class_names

YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

### Detección de coches y personas

In [1]:
from ultralytics import YOLO
import cv2

# Cargar el modelo YOLO
model = YOLO("yolov8n.pt")  # Cambia a "yolo11n-seg.pt" si es el modelo que deseas usar

# Ruta del video de entrada y salida
video_path = "./video-samples/12066183_2320_1080_30fps.mp4"  # Cambia esta ruta al video que deseas procesar
output_path = "output_coches_personas.mp4"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Clases a detectar: 0 = persona, 2 = coche, 5 = autobús, 7 = camión
target_classes = [0, 2, 5, 7]  # Clases de interés: persona, coche, autobús y camión

# Colores para cada clase
colors = {
    0: (0, 255, 0),     # Persona - Verde
    2: (0, 0, 255),    # Coche - Azul
    5: (255, 0, 0),      # Autobús - Rojo
    7: (255, 255, 0)    # Camión - Amarillo
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar la detección en el cuadro actual en la GPU
    results = model.predict(source=frame, imgsz=640, conf=0.4, device="cuda")
    
    # Procesar las detecciones
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo vehículos y personas
            if class_id in target_classes:
                # Obtener las coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Asignar el color según la clase
                color = colors.get(class_id, (0, 255, 0))  # Color por defecto: Verde

                # Dibujar el cuadro delimitador y etiqueta
                label = model.names[class_id]  # Nombre de la clase (person, car, etc.)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Cuadro de color específico
                cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Guardar el cuadro procesado en el video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()



0: 320x640 12 persons, 1 car, 1 truck, 1 backpack, 44.4ms
Speed: 3.3ms preprocess, 44.4ms inference, 108.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 persons, 2 cars, 1 truck, 7.2ms
Speed: 2.7ms preprocess, 7.2ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 persons, 1 car, 1 backpack, 8.2ms
Speed: 1.8ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 persons, 2 cars, 1 bus, 7.8ms
Speed: 2.5ms preprocess, 7.8ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 1 car, 1 truck, 6.6ms
Speed: 1.8ms preprocess, 6.6ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 2 cars, 6.9ms
Speed: 2.5ms preprocess, 6.9ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 2 cars, 7.9ms
Speed: 1.6ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 32

### Detección con IDs

In [10]:
from ultralytics import YOLO
import cv2
import numpy as np
import csv

# Cargar el modelo YOLO
model = YOLO("yolov8n.pt")

# Ruta del video de entrada y salida
video_path = "./video-samples/12066183_2320_1080_30fps.mp4"
output_path = "output_coches_personas_tracked.mp4"
csv_output_path = "detections_tracking.csv"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Clases a detectar
target_classes = [0, 2, 5, 7]

# Colores para cada clase
colors = {
    0: (0, 255, 0),     # Persona - Verde
    2: (255, 0, 0),     # Coche - Azul
    5: (0, 0, 255),     # Autobús - Rojo
    7: (255, 255, 0)    # Camión - Amarillo
}

# Diccionario de objetos rastreados {obj_id: [x1, y1, x2, y2, clase]}
tracked_objects = {}
next_object_id = 0  # ID único para cada objeto rastreado

# Diccionario para contar cada clase detectada
class_counts = {class_id: 0 for class_id in target_classes}

# Inicializar el archivo CSV y escribir el encabezado
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        "fotograma", "tipo_objeto", "confianza", "identificador_tracking",
        "x1", "y1", "x2", "y2", "matrícula_en_su_caso", "confianza_matricula",
        "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])

def calculate_iou(box1, box2):
    """Calcula el IOU entre dos cuadros delimitadores."""
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2

    # Calcular las coordenadas de la intersección
    xi1, yi1 = max(x1, x1_p), max(y1, y1_p)
    xi2, yi2 = min(x2, x2_p), min(y2, y2_p)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Calcular las áreas de los cuadros y la unión
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)
    union_area = box1_area + box2_area - inter_area

    # Calcular el IOU
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

frame_count = 0  # Contador de fotogramas

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Realizar la detección en el cuadro actual en la GPU
    results = model.predict(source=frame, imgsz=640, conf=0.4, device="cuda")

    # Almacenar las detecciones actuales
    current_detections = []

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo vehículos y personas
            if class_id in target_classes:
                # Obtener las coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                current_detections.append((x1, y1, x2, y2, class_id, confidence))

    # Asignar IDs a las detecciones usando IOU
    updated_tracked_objects = {}
    for (x1, y1, x2, y2, class_id, confidence) in current_detections:
        best_match_id = None
        highest_iou = 0.5  # Umbral mínimo para considerar que es el mismo objeto

        # Buscar el mejor ID de coincidencia basado en IOU
        for obj_id, (prev_x1, prev_y1, prev_x2, prev_y2, prev_class_id) in tracked_objects.items():
            if class_id == prev_class_id:  # Comparar solo objetos de la misma clase
                iou = calculate_iou((x1, y1, x2, y2), (prev_x1, prev_y1, prev_x2, prev_y2))
                if iou > highest_iou:
                    best_match_id = obj_id
                    highest_iou = iou

        # Si encontramos una coincidencia, actualizarla; de lo contrario, crear un nuevo ID
        if best_match_id is not None:
            updated_tracked_objects[best_match_id] = (x1, y1, x2, y2, class_id)
        else:
            updated_tracked_objects[next_object_id] = (x1, y1, x2, y2, class_id)
            best_match_id = next_object_id
            next_object_id += 1

            # Aumentar el conteo para la clase detectada
            class_counts[class_id] += 1

        # Dibujar el cuadro delimitador y el ID de seguimiento
        color = colors.get(class_id, (0, 255, 0))  # Color basado en la clase
        label = model.names[class_id]  # Nombre de la clase
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} ID: {best_match_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Escribir la información en el archivo CSV
        with open(csv_output_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                frame_count, model.names[class_id], confidence, best_match_id,
                x1, y1, x2, y2, "", "", "", "", "", ""
            ])

    # Actualizar los objetos rastreados
    tracked_objects = updated_tracked_objects

    # Guardar el cuadro procesado en el video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()

# Mostrar el conteo total de cada clase detectada
for class_id, count in class_counts.items():
    class_name = model.names[class_id]
    print(f"Total de '{class_name}': {count}")



0: 320x640 12 persons, 1 car, 1 truck, 1 backpack, 9.4ms
Speed: 4.5ms preprocess, 9.4ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 persons, 2 cars, 1 truck, 9.6ms
Speed: 2.5ms preprocess, 9.6ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 persons, 1 car, 1 backpack, 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 persons, 2 cars, 1 bus, 6.4ms
Speed: 1.6ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 1 car, 1 truck, 7.1ms
Speed: 2.4ms preprocess, 7.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 2 cars, 6.3ms
Speed: 1.8ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 2 cars, 7.0ms
Speed: 1.8ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x64

## Detector de placas

In [2]:
! yolo detect train model=yolo11n.pt data="/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/data/data.yaml" imgsz=416 batch=8 device=cuda epochs=100 patience=20  


Ultralytics 8.3.15 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5751MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/data/data.yaml, epochs=100, time=None, patience=20, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

In [6]:
from ultralytics import YOLO
import cv2
model = YOLO('/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/runs/detect/train2/weights/last.pt')
model.fuse() #model.summary 
class_names = model.model.names
class_names

YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


{0: 'd_license_plate', 1: 'pl_license_plate'}

In [7]:
from ultralytics import YOLO
import cv2


# Ruta del video de entrada y salida
video_path = "./cars.mp4"  # Cambia esta ruta al video que deseas procesar
output_path = "output_video.mp4"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Clases a detectar: 0 = d_license_plate, 1 = pl_license_plate
target_classes = [0, 1]  # Clases de interés: diferentes tipos de matrículas

# Diccionario de nombres y colores para cada clase
class_names = {0: 'd_license_plate', 1: 'pl_license_plate'}
colors = {
    0: (255, 0, 0),  # d_license_plate - Verde
    1: (255, 0, 0)   # pl_license_plate - Rojo
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar la detección en el cuadro actual en la GPU
    results = model.predict(source=frame, imgsz=640, conf=0.4, device="cuda")
    
    # Procesar las detecciones
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo clases de interés (d_license_plate y pl_license_plate)
            if class_id in target_classes:
                # Obtener las coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Asignar el color y nombre de la clase
                color = colors.get(class_id, (0, 255, 0))  # Color predeterminado: Verde
                label = class_names.get(class_id, "Unknown")  # Nombre de la clase

                # Dibujar el cuadro delimitador y etiqueta
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Cuadro de color específico
                cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Guardar el cuadro procesado en el video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()



0: 384x640 1 pl_license_plate, 48.1ms
Speed: 4.4ms preprocess, 48.1ms inference, 92.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.1ms
Speed: 2.4ms preprocess, 10.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 d_license_plate, 9.1ms
Speed: 3.3ms preprocess, 9.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pl_license_plates, 9.7ms
Speed: 3.3ms preprocess, 9.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pl_license_plates, 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pl_license_plate, 9.9ms
Speed: 3.0ms preprocess, 9.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 d_license_plate, 1 pl_license_plate, 9.4ms
Speed: 2.4ms preprocess, 9.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 d_license_plate, 1 pl_li

In [ ]:
from ultralytics import YOLO
import cv2
model = YOLO('/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/runs/detect/train2/weights/last.pt')
model.fuse() #model.summary 
class_names = model.model.names
class_names

In [7]:
! yolo detect train model=yolo11n.pt data="/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/data2/data.yaml" imgsz=416 batch=8 device=cuda epochs=50 patience=20  

Ultralytics 8.3.15 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5751MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/data2/data.yaml, epochs=50, time=None, patience=20, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

### OCR


In [8]:
from ultralytics import YOLO
import cv2
import easyocr

# Inicializa el modelo YOLO y el lector de EasyOCR
model = YOLO('/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/runs/detect/train2/weights/last.pt')
reader = easyocr.Reader(['en'])  # Puedes añadir otros idiomas si es necesario

# Ruta del video de entrada y salida
video_path = "./cars.mp4"  # Cambia esta ruta al video que deseas procesar
output_path = "output_video.mp4"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Clases a detectar: 0 = d_license_plate, 1 = pl_license_plate
target_classes = [0, 1]  # Clases de interés: diferentes tipos de matrículas

# Diccionario de nombres y colores para cada clase
class_names = {0: 'd_license_plate', 1: 'pl_license_plate'}
colors = {
    0: (255, 0, 0),  # d_license_plate - Azul
    1: (255, 0, 0)   # pl_license_plate - Rojo
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar la detección en el cuadro actual en la GPU
    results = model.predict(source=frame, imgsz=640, conf=0.4, device="cuda")
    
    # Procesar las detecciones
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo clases de interés (d_license_plate y pl_license_plate)
            if class_id in target_classes:
                # Obtener las coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Asignar el color y nombre de la clase
                color = colors.get(class_id, (0, 255, 0))  # Color predeterminado: Verde
                label = class_names.get(class_id, "Unknown")  # Nombre de la clase

                # Dibujar el cuadro delimitador y etiqueta
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si la clase es una matrícula, intenta leer el texto dentro del cuadro
                if class_id in [0, 1]:  # Detectar texto en ambas clases de matrículas
                    # Recorta el área de la matrícula del cuadro
                    license_plate = frame[y1:y2, x1:x2]
                    
                    # Realiza el reconocimiento de texto en la imagen de la matrícula
                    results_ocr = reader.readtext(license_plate)
                    for detection in results_ocr:
                        text = detection[1]  # El texto detectado
                        print("Matrícula detectada:", text)  # Imprime el texto de la matrícula (opcional)

                        # Dibuja el texto en el cuadro de la matrícula
                        cv2.putText(frame, text, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Guardar el cuadro procesado en el video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete
0: 384x640 1 pl_license_plate, 11.1ms
Speed: 2.1ms preprocess, 11.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Matrícula detectada: Wsisu

0: 384x640 (no detections), 9.6ms
Speed: 3.5ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 d_license_plate, 9.5ms
Speed: 2.3ms preprocess, 9.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Matrícula detectada: Nai3 NRI

0: 384x640 2 pl_license_plates, 9.2ms
Speed: 2.6ms preprocess, 9.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Matrícula detectada: NAI3 NRU
Matrícula detectada: WEiSU

0: 384x640 2 pl_license_plates, 9.6ms
Speed: 3.3ms preprocess, 9.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Matrícula detectada: NIJNRU
Matrícula detectada: OsSUt

0: 384x640 1 pl_license_plate, 9.3ms
Speed: 2.1ms preprocess, 9.3ms inference, 1.3

In [1]:
from ultralytics import YOLO
import cv2
import easyocr

# Inicializa el modelo YOLOv8 para detección de personas y coches y el modelo para detección de matrículas
yolo_model_person_car = YOLO('yolov8n.pt')  # Modelo YOLOv8 para personas y coches (modifica si usas otro archivo)
license_plate_model = YOLO('/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/runs/detect/train2/weights/last.pt')
reader = easyocr.Reader(['en'])  # Idioma inglés para OCR

# Ruta del video de entrada y salida
video_path = "/home/0xm4rc/Downloads/C0142.MP4"
output_path = "output_video_with_license_plates_final.mp4"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Clases a detectar: personas y coches en YOLOv8, matrículas en el modelo de detección de matrículas
person_car_classes = [0, 2]  # ID de persona y coche en YOLOv8 (según la configuración del modelo)
target_classes_license = [0, 1]  # Matrículas en el modelo de detección de matrículas

# Colores y nombres de clases para cada modelo
class_names_yolo = {0: 'person', 2: 'car'}
class_names_license = {0: 'd_license_plate', 1: 'pl_license_plate'}
colors_yolo = {
    0: (0, 255, 0),   # Verde para personas
    2: (255, 0, 0)    # Azul para coches
}
colors_license = {
    0: (255, 0, 0),  # Azul para d_license_plate
    1: (0, 0, 255)   # Rojo para pl_license_plate
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de personas y coches en el cuadro actual usando YOLOv8
    results_person_car = yolo_model_person_car.predict(source=frame, imgsz=640, conf=0.4, device="cuda")
    
    # Procesa detecciones de personas y coches
    for result in results_person_car:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase detectada
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo personas y coches
            if class_id in person_car_classes:
                # Obtener las coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Asignar color y nombre de clase según el ID
                color = colors_yolo.get(class_id, (0, 255, 255))  # Color predeterminado: Amarillo
                label = class_names_yolo.get(class_id, "Unknown")  # Nombre de la clase

                # Dibujar el cuadro delimitador y etiqueta en la detección de personas y coches
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si se detecta un coche, pasa el recorte a la función de detección de matrículas
                if class_id == 2:  # ID de coche
                    car_region = frame[y1:y2, x1:x2]

                    # Detección de matrículas en la región del coche
                    results_license = license_plate_model.predict(source=car_region, imgsz=640, conf=0.4, device="cuda")
                    for license_result in results_license:
                        for license_box in license_result.boxes:
                            license_class_id = int(license_box.cls[0])
                            license_conf = license_box.conf[0]

                            # Filtrar solo clases de matrículas
                            if license_class_id in target_classes_license:
                                # Coordenadas del cuadro delimitador de la matrícula
                                lx1, ly1, lx2, ly2 = map(int, license_box.xyxy[0])

                                # Ajustar las coordenadas de la matrícula a la posición en el marco principal
                                lx1, lx2 = lx1 + x1, lx2 + x1
                                ly1, ly2 = ly1 + y1, ly2 + y1

                                # Color y etiqueta de la matrícula
                                license_color = colors_license.get(license_class_id, (0, 255, 255))  # Color por clase
                                license_label = class_names_license.get(license_class_id, "Unknown")

                                # Dibuja el cuadro delimitador de la matrícula en el marco principal
                                cv2.rectangle(frame, (lx1, ly1), (lx2, ly2), license_color, 2)
                                cv2.putText(frame, f"{license_label} {license_conf:.2f}", (lx1, ly1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, license_color, 2)

                                # Recortar la imagen de la matrícula para OCR
                                license_plate_img = frame[ly1:ly2, lx1:lx2]

                                # Detección de texto en la matrícula usando EasyOCR
                                results_ocr = reader.readtext(license_plate_img)
                                for detection in results_ocr:
                                    text = detection[1]  # Texto de la matrícula
                                    print("Matrícula detectada:", text)  # Muestra el texto (opcional)

                                    # Dibuja el texto detectado en el cuadro de la matrícula
                                    cv2.putText(frame, text, (lx1, ly2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Guardar el cuadro procesado en el video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()



0: 384x640 1 person, 3 cars, 1 bus, 1 bench, 49.3ms
Speed: 3.3ms preprocess, 49.3ms inference, 119.3ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 (no detections), 47.6ms
Speed: 1.9ms preprocess, 47.6ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 (no detections), 43.4ms
Speed: 1.9ms preprocess, 43.4ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 (no detections), 44.1ms
Speed: 0.8ms preprocess, 44.1ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 person, 3 cars, 1 bus, 1 bench, 7.2ms
Speed: 2.3ms preprocess, 7.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 (no detections), 12.2ms
Speed: 1.4ms preprocess, 12.2ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 (no detections), 12.2ms
Speed: 1.8ms preprocess, 12.2ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 (no detections), 12.0ms

In [5]:
from ultralytics import YOLO
import cv2
import easyocr
import pandas as pd

# Inicialización de modelos
yolo_model_person_car = YOLO('yolov8n.pt')  # Modelo YOLOv8 para detección de personas y coches
license_plate_model = YOLO('/home/0xm4rc/Desktop/VC/Ejercicios Practicas/Practica4/runs/detect/train2/weights/last.pt')
reader = easyocr.Reader(['en'])  # OCR en inglés

# Configuración de video y CSV
video_path = "/home/0xm4rc/Downloads/C0142.MP4"
output_video_path = "output_video_with_license_plates_final.mp4"
output_csv_path = "detections_tracking.csv"

# Configuración de captura de video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Parámetros para conteo y almacenamiento de resultados
data = []
object_counts = {"person": 0, "car": 0}

# Diccionario para nombres de clases y colores
class_names_yolo = {0: 'person', 2: 'car'}
class_names_license = {0: 'd_license_plate', 1: 'pl_license_plate'}
colors_yolo = {
    0: (0, 255, 0),   # Verde para personas
    2: (255, 0, 0)    # Azul para coches
}
colors_license = {
    0: (255, 0, 0),  # Azul para d_license_plate
    1: (0, 0, 255)   # Rojo para pl_license_plate
}

# Inicialización de identificador de seguimiento
tracker_id_counter = 0
trackers = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Detección de personas y coches usando YOLOv8
    results_person_car = yolo_model_person_car.predict(source=frame, imgsz=640, conf=0.4, device="cuda")

    # Procesa detecciones de personas y coches
    for result in results_person_car:
        for box in result.boxes:
            class_id = int(box.cls[0])  # ID de la clase detectada
            confidence = box.conf[0]    # Confianza de la detección

            # Filtrar solo personas y coches
            if class_id in class_names_yolo.keys():
                # Obtener coordenadas del cuadro delimitador
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Asignar color y nombre de clase
                color = colors_yolo[class_id]
                label = class_names_yolo[class_id]
                object_counts[label] += 1

                # Dibuja el cuadro y etiqueta en el video
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Generar identificador de seguimiento
                tracker_id = trackers.get((x1, y1, x2, y2), tracker_id_counter)
                if tracker_id == tracker_id_counter:
                    tracker_id_counter += 1
                trackers[(x1, y1, x2, y2)] = tracker_id

                # Si se detecta un coche, se realiza detección de matrícula
                if class_id == 2:
                    car_region = frame[y1:y2, x1:x2]
                    results_license = license_plate_model.predict(source=car_region, imgsz=640, conf=0.4, device="cuda")

                    # Procesa detección de matrículas
                    for license_result in results_license:
                        for license_box in license_result.boxes:
                            license_class_id = int(license_box.cls[0])
                            license_conf = license_box.conf[0]

                            # Filtrar solo clases de matrículas
                            if license_class_id in class_names_license.keys():
                                lx1, ly1, lx2, ly2 = map(int, license_box.xyxy[0])
                                lx1, lx2 = lx1 + x1, lx2 + x1
                                ly1, ly2 = ly1 + y1, ly2 + y1

                                license_color = colors_license[license_class_id]
                                license_label = class_names_license[license_class_id]

                                # Dibuja el cuadro de la matrícula y etiqueta
                                cv2.rectangle(frame, (lx1, ly1), (lx2, ly2), license_color, 2)
                                cv2.putText(frame, f"{license_label} {license_conf:.2f}", (lx1, ly1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, license_color, 2)

                                # OCR para la matrícula
                                license_plate_img = frame[ly1:ly2, lx1:lx2]
                                results_ocr = reader.readtext(license_plate_img)
                                detected_text = ""
                                if results_ocr:
                                    detected_text = results_ocr[0][1]
                                    print("Matrícula detectada:", detected_text)
                                    cv2.putText(frame, detected_text, (lx1, ly2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                                # Almacenar datos de detección y seguimiento en la lista de datos
                                data.append({
                                    "fotograma": frame_number,
                                    "tipo_objeto": label,
                                    "confianza": confidence,
                                    "identificador_tracking": tracker_id,
                                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                                    "matrícula_en_su_caso": license_label if license_class_id in [0, 1] else None,
                                    "confianza_matrícula": license_conf if license_class_id in [0, 1] else None,
                                    "mx1": lx1 if license_class_id in [0, 1] else None,
                                    "my1": ly1 if license_class_id in [0, 1] else None,
                                    "mx2": lx2 if license_class_id in [0, 1] else None,
                                    "my2": ly2 if license_class_id in [0, 1] else None,
                                    "texto_matricula": detected_text if license_class_id in [0, 1] else None
                                })

    # Escribe el cuadro procesado en el video de salida
    out.write(frame)

# Guardar resultados en CSV
df = pd.DataFrame(data)
df.to_csv(output_csv_path, index=False)

# Liberar recursos
cap.release()
out.release()

print(f"Conteo total de objetos: {object_counts}")
print(f"Resultados guardados en {output_csv_path}")
print("Proceso completado.")



0: 384x640 1 person, 3 cars, 1 bus, 1 bench, 10.1ms
Speed: 1.6ms preprocess, 10.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 (no detections), 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 (no detections), 12.6ms
Speed: 2.2ms preprocess, 12.6ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 (no detections), 13.5ms
Speed: 1.7ms preprocess, 13.5ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 person, 3 cars, 1 bus, 1 bench, 6.8ms
Speed: 2.8ms preprocess, 6.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 (no detections), 10.2ms
Speed: 1.8ms preprocess, 10.2ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 (no detections), 13.3ms
Speed: 2.0ms preprocess, 13.3ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 (no detections), 10.6ms
Spe

KeyboardInterrupt: 